In [55]:
import json
import pandas as pd
from pathlib import Path
import re
import os

In [56]:
# data_json = Path('falcon_links') / '1ents_1rels' / 'link_28246.json'
# data_json = Path('falcon_links') / '5ents_5rels' / 'link_24066.json'
# data_json = Path('falcon_links') / '1ents-gold_0rels' / 'link_28246.json'
# data_json = Path('falcon_links') / '5ents-gold_0rels' / 'link_28246.json'
# data_json = Path('falcon_links') / '10ents-gold_0rels' / 'link_28246.json'
# data_json = Path('falcon_links') / '10ents_10rels' / 'link_28246.json'
data_json = Path('falcon_links') / '5ents-gold_5rels-gold' / 'link_28246.json'
# data_json = Path('falcon_links') / '10ents-gold_10rels-gold' / 'link_28246.json'

with open(data_json) as f:
  data = json.load(f)


In [57]:
data_filtered = []

for entry in data:

  linked, annotated, converted = entry
  link_ents = {e['id'] for e in linked['ents']}
  link_rels = {r['id'] for r in linked['rels']}
  gold_query = converted['labels']
  gold_ents = set()
  gold_rels = set()
  for tok in gold_query.split(" "):
    tok_match = re.search('[QP][0-9]+', tok.upper())
    if not tok_match:
      continue
    tok = tok_match.group(0)
    if tok[0].upper() == "Q":
      gold_ents.add(tok.upper())
    elif tok[0].upper() == "P":
      gold_rels.add(tok.upper())
  res = {
      "utterance": linked["utterance"],
      "link_ents": link_ents,
      "gold_ents": gold_ents,
      "link_rels": link_rels,
      "gold_rels": gold_rels,
  }
  data_filtered.append(res)

In [58]:
results = []
import re

def recall(real, pred):
  # real = list(filter(lambda f:re.search('[QP][0-9]+', f), real))
  if len(real) == 0:
    return 1.0
  count = 0
  for x in real:
    if x in pred:
      count += 1 
  # if count / len(real) < 0.99:
  #   print(real, pred)
  return count/len(real)

for i, data in enumerate(data_filtered):
  ent_recall = recall(data["gold_ents"], data["link_ents"])
  rel_recall = recall(data["gold_rels"], data["link_rels"])
  
  # if ent_recall < 0.99 :
  #   print(data)

  results_dict= {
    "utterance": data["utterance"],
    "ent_recall": ent_recall,
    "rel_recall": rel_recall,
    "pos": i,
  }
  results.append(results_dict)

In [59]:
data_filtered[40]

{'utterance': 'Tell me the name of a fantastique genre that starts with the letter s.',
 'link_ents': {'Q1168233',
  'Q133492',
  'Q1579140',
  'Q20076756',
  'Q248887',
  'Q293618',
  'Q31198',
  'Q342812',
  'Q384031',
  'Q411839',
  'Q9788'},
 'gold_ents': {'Q20076756'},
 'link_rels': {'P118', 'P1350', 'P17', 'P31', 'P47', 'P54'},
 'gold_rels': {'P31'}}

In [60]:
df = pd.DataFrame.from_dict(results)
print("Mean Ent Recall:", df.ent_recall.mean())
print("Correct:", len(df[df.ent_recall >= 0.99]) / len(df))
print("Mean Rel Recall:", df.rel_recall.mean())
print("Correct:", len(df[df.rel_recall >= 0.99]) / len(df))
df


Mean Ent Recall: 1.0
Correct: 1.0
Mean Rel Recall: 1.0
Correct: 1.0


,utterance,ent_recall,rel_recall,pos
0,What periodical literature does Delta Air Line...,1.0,1.0,0
1,Who is the child of Ranavalona Is husband?,1.0,1.0,1
2,Is it true Jeff_Bridges occupation Lane Chandl...,1.0,1.0,2
3,What is the pre-requisite of phase matter of G...,1.0,1.0,3
4,Which is the operating income for Qantas?,1.0,1.0,4
...,...,...,...,...
28239,What borders the birth place of Johann Pachelbel?,1.0,1.0,28239
28240,What is the depiction of the narrative of Thom...,1.0,1.0,28240
28241,Is the torque of the NEMA 17 Stepper motor Mod...,1.0,1.0,28241
28242,Is Diabetes mellitus studied in diabetology?,1.0,1.0,28242


In [61]:
df[df.ent_recall < 0.99]

,utterance,ent_recall,rel_recall,pos


In [62]:
data_filtered[2682]

{'utterance': 'Which is the flag carrier which contains the word qantas in its name?',
 'link_ents': {'Q1073895',
  'Q1374036',
  'Q1392130',
  'Q14660',
  'Q2141043',
  'Q297187',
  'Q32491',
  'Q721639',
  'Q869352',
  'Q877284'},
 'gold_ents': {'Q721639'},
 'link_rels': {'P1383', 'P150', 'P31', 'P361', 'P4330', 'P527'},
 'gold_rels': {'P31'}}